# Regression Tests

This is the regression test suite for Asteroid.  All the programs in this notebook should execute.  

You can turn this into an executable Python script with:
```
jupyter nbconvert --to script regression-tests.ipynb 
```

In [1]:
from asteroid_interp import interp

In [2]:
program = \
'''
load "io".
let fst = (1,2)@0.
print fst.
'''
interp(program, exceptions=False)

1


In [3]:
program = \
'''
load "standard".
load "io".
let t = (1,2).
print ("tuple = " + t).
'''
interp(program, exceptions=True)

tuple = (1,2)


In [4]:
program = \
'''
let () = none.
'''
interp(program, exceptions=False)

In [5]:
program = \
'''
load "standard".
load "io".
load "util".

class Dog with

  data name = "".
  data tricks = [].

  function add_trick
    with (self, new_trick) do
      let self@tricks = self@tricks + [new_trick].
    end function
    
  function __init__
    with (self, name) do 
      let self@name = name.
    end function
    
  end class

-- Fido the dog
let fido = Dog("Fido").

fido@add_trick("roll over").
fido@add_trick("play dead").

-- Buddy the dog
let buddy = Dog("Buddy").

buddy@add_trick("roll over").
buddy@add_trick("sit stay").

-- print out the tricks
print ("Fido: " + fido@tricks).
print ("Buddy: " + buddy@tricks).

assert(fido@tricks is ["roll over", "play dead"]).
assert(buddy@tricks is ["roll over", "sit stay"]).
'''
interp(program, exceptions=False)

Fido: [roll over,play dead]
Buddy: [roll over,sit stay]


In [6]:
program = \
'''
load "standard".
load "io".
load "util".

class Dog with

  data name = "".
  data tricks = [].
  data add_trick = lambda with (self, new_trick) do let self@tricks = self@tricks + [new_trick].
  data __init__ = lambda with (self, name) do let self@name = name.

  end class

-- Fido the dog
let fido = Dog("Fido").

fido@add_trick "roll over".
fido@add_trick "play dead".

-- Buddy the dog
let buddy = Dog("Buddy").

buddy@add_trick "roll over".
buddy@add_trick "sit stay".

-- print out the tricks
print ("Fido: " + fido@tricks).
print ("Buddy: " + buddy@tricks).

assert(fido@tricks is ["roll over", "play dead"]).
assert(buddy@tricks is ["roll over", "sit stay"]).

'''
interp(program)

Fido: [roll over,play dead]
Buddy: [roll over,sit stay]


In [7]:

program = \
'''
load "io".
load "util".

class A 
  with
    data x = 2.
    function dump with self do print self@x end function
  end class

let obj = A().
obj@dump().

let A(x,y) = obj.
print x.

assert(x is 2).

'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

2
2


In [8]:
program = \
'''
load "io".
load "standard".
load "util".


constructor A with arity 1.

let a = A(lambda with (a,b) do return a+b).

print a@0 (1,2).

assert(a@0(1,2) is 3).

'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

3


In [9]:
program = \
'''
load "io".
load "util".

class A with

    data x.
    data y.

    function __init__
      with (self, a, b) do
        let self@x = a.
        let self@y = b.
      orwith self do
        let self@x = 1.
        let self@y = 2.
      end function
    end class

let obj1 = A("hello","world").
print obj1.
let obj2 = A(none).
print obj2.

assert((obj1@x is "hello") and (obj1@y is "world")).
assert((obj2@x is 1) and (obj2@y is 2)).

'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=True, do_walk=True)

A(hello, world, (function ...))
A(1, 2, (function ...))


In [10]:
program =\
'''
load "io".
load "util".

-- color to hex conversion dictionary
let colors = [
    ("red", "FF,00,00"),
    ("green", "00,FF,00"),
    ("yellow", "FF,FF,00")
    ].
    
print (colors@{"red"}).

assert(colors@{"red"} is "FF,00,00").
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

FF,00,00


In [11]:
program = \
'''
load "standard".
load "io".
load "util".

constructor Person with arity 3.

let people = [
    Person("George", 32, "M"),
    Person("Sophie", 46, "F"),
    Person("Oliver", 21, "M")
    ].
    
let Person(name,age,sex) = people@1.
let output_str = name + " is " + age + " years old and is " + "male" if sex is "M" else "female" + ".".
print output_str.
assert(output_str is "Sophie is 46 years old and is female.")
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

Sophie is 46 years old and is female.


In [12]:
program = \
"""
load "standard".
load "io". 
load "util".
let l = [("test", "testing")].
let a = l@{"test"}.
print a.
assert(a is "testing").
"""
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

testing


In [13]:
from asteroid_interp import interp
program = \
"""
load "standard".
load "io".      
load "util".
let slice = [4 to 0 step -1].
print slice.
assert(slice is [4,3,2,1,0]).
"""
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

[4,3,2,1,0]


In [14]:
program = \
'''
load "standard".
load "io".
load "util".

let a = [[1,2],[3,4],[5,6]].
let l = (a@0).
print l.
assert(l is [1,2]).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

[1,2]


In [15]:
program = \
'''
load "io".
load "util".

let a = [10,20,30].
let x = a@(1).
let y = a@[1].

print x. -- prints out a scalar 
print y. -- prints out a list
assert((x is 20) and (y is [20])).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

20
[20]


In [16]:
program = \
'''
load "standard"
load "util".
load "io".

let v = random none.

print v.
assert(v <= 1.0 and v >= 0.0).

'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False, do_walk=True)

0.9654879291515361


In [17]:
program = \
'''
load "standard"
load "io".
load "util".

let inc = (lambda with n do return n+1).


print (eval ('inc 1)).

assert(eval ('inc 1) is 2).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=True)

2


In [18]:

program = \
'''
load "io".
load "util".
let (f,g) = (1,2).

function foobar 
    with none do
        global f, g.
        let f = 2.
    end function
    
foobar(none).
print (f,g).

assert((f,g) is (2,2)).
'''
interp(program, tree_dump=False, do_walk=True, exceptions=False, symtab_dump=False)

(2,2)


In [19]:
program = \
'''
load "io".

try 
    let 2 = 1 + 1. 
catch _ do
    print "pattern match failed".
end try

'''
interp(program, tree_dump=False, do_walk=True, exceptions=False, symtab_dump=False)

pattern match failed


In [20]:
program = \
'''
load "io".

load "default". -- load default term model
let 1 + 1 = 1 + 1. -- (*@\label{patternmatching-models:let1}@*)
try 
    let 2 = 1 + 1. -- (*@\label{patternmatching-models:let1a}@*)
catch _ do
    print "pattern match failed".
end try

load "standard". -- load standard model (*@\label{patternmatching-models:load}@*)
let 2 = 1 + 1. -- (*@\label{patternmatching-models:let2}@*)
try 
    let 1 + 1 = 1 + 1. -- throws an exception (*@\label{patternmatching-models:let3}@*)
catch _ do
    print "pattern match failed".
end try

'''
interp(program, tree_dump=False, do_walk=True, exceptions=True, symtab_dump=False)

pattern match failed
pattern match failed


In [21]:
program = \
'''
-- TODO: this pattern match fails under the standard model
--load "standard".
let -1 = -1 .
'''
interp(program, tree_dump=False, do_walk=True, exceptions=False, symtab_dump=False)

In [22]:
program = \
'''
load "standard".
load "io".
load "util".
let p = -1.
let w = p - 1.
let q = [-1].
print (p,w,q).

-- NOTE: workaround, pattern matching does not work on negative constants.
assert(p == -1).
assert(w == -2).
assert(q == [-1]).

'''
interp(program, tree_dump=False, do_walk=True, exceptions=False, symtab_dump=False)

(-1,-2,[-1])


In [23]:
program = \
'''
-- overload the model
load "math". -- defines mult_op
load "standard".
load "io".    
load "util".

function dot_op
    with ([a],[b]) do
        return mult_op(a,b).
    orwith ([h1|t1],[h2|t2]) do
        return mult_op(h1,h2) + dot_op(t1,t2).
    end function
    
function mult_op_overloaded
    with (v1,v2) do
        if is_list(v1) and is_list(v2) do
            return dot_op(v1,v2).
        else do
            return mult_op(v1,v2).
        end if
    end function

detach from __times__.
attach  mult_op_overloaded to __times__.

print ("[2,2,2]*[2,2,2]=" + [2,2,2]*[2,2,2]).
print ("2*2=" + 2*2).

assert([2,2,2]*[2,2,2] == 12 and 2*2 == 4).
'''
interp(program, symtab_dump=False, exceptions=True)

[2,2,2]*[2,2,2]=12
2*2=4


In [24]:
program = \
'''
load "io".
load "util".

constructor S with arity 1.

function inc 
    with n do
        return 1 + n.
    end function

print (S(S(S(0)))).
assert (S(S(S(0))) is S(S(S(0)))).

attach inc to S.
print (S(S(S(0)))).
assert (S(S(S(0))) is __plus__(1,__plus__(1,__plus__(1,0)))).
'''
interp(program,exceptions=False)

S(S(S(0)))
__plus__(1,__plus__(1,__plus__(1,0)))


In [25]:
program = \
'''
load "standard".
load "io".
load "util".

constructor S with arity 1.

function inc 
    with n do
        return 1 + n.
    end function

-- switch between models
print (S(S(S(0)))).
assert (S(S(S(0))) is S(S(S(0)))).

attach inc to S.
print (S(S(S(0)))).
assert (S(S(S(0))) is 3).

detach from S.
print (S(S(S(0)))).
assert (S(S(S(0))) is S(S(S(0)))).
'''
interp(program,exceptions=False)

S(S(S(0)))
3
S(S(S(0)))


In [26]:
program = \
'''
-- implements Peano addition using a lookup table for the rewrite rules

load "standard".
load "util".
load "io".

detach from __plus__ . -- we want to use '+' as constructor
constructor S with arity 1.

let rule_table = [
    ('x + 0, 'reduce(x)),
    ('x + S(y), 'S(reduce(x + y)))
    ].

function reduce 
    with term do
        for i in 0 to length(rule_table) - 1 do
            -- limit visibility of free variables of the rewrite rules
            -- to the with block scope
            let (lhs, rhs) = rule_table@i.
            if term is *lhs do
                return eval rhs.
            end if
        end for
        return term.
    end function

print (reduce(S(S(0)) + S(S(S(0))))).
assert (reduce(S(S(0)) + S(S(S(0)))) is S(S(S(S(S(0)))))).
'''

interp(program,exceptions=False,symtab_dump=False, tree_dump=False, do_walk=True)

S(S(S(S(S(0)))))


In [27]:
program = \
'''
load "io".
load "util".

let cl = 1 + 2.
let cr = 3.
let pattern = cl + cr.

print (1+2+3 is *pattern).
assert ((1+2+3 is *pattern) is true).
'''
interp(program, tree_dump=False, exceptions=False)

true


In [28]:
program = \
'''
load "io".
load "util".

function match
    with (subject, pattern) do
        return subject is *pattern.
    end function
    
print (match(1+1, '_+_)).
assert (match(1+1, '_+_) is true).
'''
interp(program, tree_dump=False, exceptions=False)

true


In [29]:
program = \
'''
load "io".

constructor MyException with arity 1.

try
    throw MyException("Hello There!").
catch MyException(v) do
    print v.
end try
'''
interp(program, tree_dump=False, exceptions=False)

Hello There!


In [30]:
program = \
'''
load "io".
load "standard".

try
    let i = 10/0.
    print i.
catch e do
    print e.
end try
'''
interp(program, tree_dump=False, exceptions=False)

(Exception,integer division or modulo by zero)


In [31]:
program = \
'''
load "io".
load "standard".

try
    let i = 10/0.
    print i.
catch ("Exception", v) do
    print v.
end try
'''
interp(program, tree_dump=False, exceptions=False)

integer division or modulo by zero


In [32]:
program = \
'''
load "standard".
load "io".
load "util".

constructor Person with arity 3.

let people = [
    Person("George", 32, "M"),
    Person("Sophie", 46, "F"),
    Person("Oliver", 21, "M")
    ].
    
let n = length people.
let sum = 0.

for Person(_,age,_) in people do
    let sum = sum + age.
end for

print ("Average Age: " + (sum/n)).
assert ((sum/n) == 33).

for Person(name,_,"M") in people do
    print name.
end for
'''
interp(program, tree_dump=False, exceptions=False)

Average Age: 33
George
Oliver


In [33]:
program = \
'''
load "io".

let list = [1,2,3].

repeat do
    let [head|list] = list.
    print head.
until list is [_].
'''
interp(program, tree_dump=False, exceptions=False)

1
2


In [34]:
program = \
'''
load "io".
load "util".

let true = 1 + 2 is x + y.
print (x,y).
assert ((x,y) is (1,2)).
'''
interp(program, tree_dump=False, exceptions=False)

(1,2)


In [35]:
program = \
'''
load "util".
constructor S with arity 1.

let v = S(S(S(0))).
assert(v is S(S(S(0)))).
'''
interp(program, tree_dump=False, exceptions=False)

In [36]:
program = \
'''
-- implements Peano addition on terms
load "io".
load "util".

constructor S with arity 1.

function reduce
    with x + 0 do      
        return reduce(x).
    orwith x + S(y)  do
        return S(reduce(x + y)).
    orwith term do     
        return term.
    end function

print(reduce(S(S(0)) + S(S(S(0))) + S(0) + S(0))).
assert((reduce(S(S(0)) + S(S(S(0))) + S(0) + S(0))) is S(S(S(S(S(S(S(0)))))))).
'''
interp(program, tree_dump=False, exceptions=False)

S(S(S(S(S(S(S(0)))))))


In [37]:
program = \
'''
-- implements Peano addition on terms using pattern matching in if statements
load "io".
load "util".

constructor S with arity 1.

function reduce 
    with term do
        if term is x + 0 do      
            return reduce(x).
        elif term is x + S(y)  do
            return S(reduce(x + y)).
        else     
            return term.
        end if
    end function

print(reduce(S(S(0)) + S(S(S(0))) + S(0) + S(0))).
assert((reduce(S(S(0)) + S(S(S(0))) + S(0) + S(0))) is S(S(S(S(S(S(S(0)))))))).
'''
interp(program, tree_dump=False, exceptions=False)

S(S(S(S(S(S(S(0)))))))


In [38]:
program = \
'''
-- Quicksort

load "standard".
load "io".
load "util".

function qsort
    with [] do
        return [].
    orwith [a] do
        return [a].
    orwith [pivot|rest] do
        let less=[].
        let more=[].
            
        for e in rest do  
            if e < pivot do
                let less = less + [e].
            else
                let more = more + [e].
            end if
        end for
                     
        return qsort less + [pivot] + qsort more.
    end function
    
print (qsort [3,2,1,0]).
assert (qsort [3,2,1,0] == [0,1,2,3]).
'''
interp(program, symtab_dump=False)

[0,1,2,3]


In [39]:
program = \
'''
load "standard".
load "io".
load "util".

let 1 + 1 = '1 + 1. -- quoted expression
let 2 = eval('1 + 1).
let 2 = 1 + 1.
try
    let 1 + 1 = 1 + 1.
catch ("PatternMatchFailed", v) do
    print v.
end try

'''
interp(program, tree_dump=False, do_walk=True, exceptions=False, symtab_dump=False)

pattern match failed: term and pattern disagree on 'apply-list' node


In [40]:
program = \
'''
load "standard".
load "io".
load "util".

attach (lambda with (a, b) do return a * b) to __plus__.

print (3 + 2).  
assert ((3 + 2) == 6).  

detach from __plus__.
print (3 + 2).  
assert ((3 + 2) == 5).  
'''
interp(program, symtab_dump=False)

6
5


In [41]:
program = \
'''
load "io".
load "util".

function postfix
    with (op, cl, cr) do 
        return (postfix cl, postfix cr, op)
    orwith (op, c) do 
        return (postfix c, op)
    orwith (v,) do 
        return (v,)
end function

print (postfix ("+", (1,), (2,))).
assert ((postfix ("+", (1,), (2,))) is ((1,),(2,),"+")).
'''
interp(program, tree_dump=False, do_walk=True, exceptions=True, symtab_dump=False)

((1,),(2,),+)


In [42]:
program = \
'''
load "io". -- load io module
load "util".

-- print out the value using the default term model
print (4 + 3 - 2).
assert ((4 + 3 - 2) is __minus__(__plus__(4,3),2)).

-- load the standard model and print out value of the term
load "standard".
print (4 + 3 - 2).
assert ((4 + 3 - 2) == 5).

-- manipulate the standard model, switching the interpretations
-- of plus and minus.

-- save the interpretations
let plus_op = __plus__.
let minus_op = __minus__.

-- detach the interpretations from constructors
detach from __plus__.
detach from __minus__.

-- reattach in opposite order and print the value of the term
attach plus_op to __minus__.
attach minus_op to __plus__.

print (4 + 3 - 2).
assert ((4 + 3 - 2) == 3).
'''
interp(program, tree_dump=False, do_walk=True, exceptions=False, symtab_dump=False)

__minus__(__plus__(4,3),2)
5
3


In [43]:
program = \
'''
-- to see the effect of detach all comment out the second line and look at the symbol table. 
load "standard".
load "default".
'''
interp(program, tree_dump=False, do_walk=True, exceptions=False, symtab_dump=False)

In [44]:
program = \
'''
load "io".
load "util".

try
    detach from __plus__.
catch ("Exception", v) do
    print v.
    assert (v is "Cannot detach constructor from __plus__.").
end try
'''
interp(program, tree_dump=False, do_walk=True, exceptions=True)

Cannot detach constructor from __plus__.


In [45]:
program = \
'''
load "io".
load "util".

try
    let 2 = 1 + 1.
catch ("PatternMatchFailed", v) do
    print v.
    assert (v is "pattern match failed: nodes 'apply-list' and 'integer' are not the same").
end try
'''
interp(program, tree_dump=False, do_walk=True, exceptions=False)

pattern match failed: nodes 'apply-list' and 'integer' are not the same


In [46]:
program = \
'''
load "io".

let 1 + 1 = 1 + 1.
load "standard". -- load standard model
let 2 = 1 + 1.
try
    let 1 + 1 = 1 + 1.
catch _ do
    print "not a legal unification under the standard model".
end try
'''
interp(program, tree_dump=False, do_walk=True)

not a legal unification under the standard model


In [47]:
program =\
'''
load "util".
load "io".

let cnt = integer(input("Please enter an integer value: ")).

for i in 1 to cnt do
    print i.
end for
assert (i is cnt).
'''

interp(program)

Please enter an integer value: 5
1
2
3
4
5


In [48]:
program =\
'''
load "standard".
load "io".

let name = input("Please enter your name: ").
print("Hello " + name + "!").
'''

interp(program)

Please enter your name: Fred
Hello Fred!


In [49]:
program = \
'''
load "util".
load "io".

constructor S with arity 1.

let x = 'S(S(0)).
let y = 'S(S(x)).
let z = y.

print y.
print z.
print (eval (z)).
assert ((eval (z)) is S(S(S(S(0))))).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

'S(S(x))
'S(S(x))
S(S('S(S(0))))


In [50]:
program = \
'''
load "io".
load "util".

function ident 
    with n do 
        return n 
    end function 

let y = ident ident  0.

print y.
assert (y is 0).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=True)

0


In [51]:
program = \
'''
load "io".
load "util".

function ident 
    with n do 
        return n 
    end function 

let y = ident(ident (0)).
let x = ident ident 0.

print (x,y).
assert ((x,y) is (0,0)).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

(0,0)


In [52]:
program = \
'''
-- Factorial

load "standard".
load "io".
load "util".

function fact 
    with 0 do
        return 1
    orwith n do
        return n * fact (n-1).
    end function

print ("The factorial of 3 is: " + fact (3)).
assert (fact(3) == 6).
'''
interp(program, exceptions=False, symtab_dump=False)

The factorial of 3 is: 6


In [53]:
program = \
'''
-- show that the value constructed by head-tail is a list
let [1,2,3] = [1 | [2,3]].

-- show that a list can be decomposed with head-tail
let [1 | [2,3]] = [1,2,3].

-- show that we can nest head-tail operators
let [1,2,3] = [1 | [2 | [3 | []]]].

'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

In [54]:
program = \
'''
load "standard".
load "util".
load "io".

constructor MyError with arity 1.


try

    throw Error "--- error ---".
    
catch Error(msg) do
    print msg.
    assert (msg == "--- error ---")
end try

'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

--- error ---


In [55]:
program = \
'''
load "standard".
load "io".
load "util".

let [h|t] = [1,2,3].
print ("head: " + h + " tail: " + t).
assert(h == 1 and t == [2,3]).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

head: 1 tail: [2,3]


In [56]:
program = \
'''
load "standard".
load "io".
load "util".

let y = -1.
let x = 4 if y == 3 else 0.
print x.
assert (x == 0).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

0


In [57]:
program = \
'''
load "io".
load "util".

let y = none.
let x = y otherwise 1.
print x.
assert (x is 1).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

1


In [58]:
program = \
'''
load "standard".
load "io".
load "util".

for x in 0 to 10 do
    print x.
    if x == 5 do
        break.
    end if
end for
assert (x == 5).
'''
interp(program, tree_dump=False, do_walk=True, exceptions=False)

0
1
2
3
4
5


In [59]:
program = \
'''
load "io".

for (x,y) in [(1,1), (2,2), (3,3)]  do
    print (x,y).
end for

-- use unification as a filter
for (2,y) in [(1,11), (1,12), (1,13), (2,21), (2,22), (2,23)]  do
    print y.
end for
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False)

(1,1)
(2,2)
(3,3)
21
22
23


In [60]:
program = \
'''
load "io".

for x in 1 to 10 do
    print x.
end for
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False)

1
2
3
4
5
6
7
8
9
10


In [61]:
program = \
'''
load "io".

for bird in ["turkey","duck","chicken"] do
    print bird.
end for
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False)

turkey
duck
chicken


In [62]:
program = \
'''
load "standard".
load "io".

let x = 42.

if x < 0 do
    let x = 0.
    print("Negative changed to zero").

elif x == 0 do
    print("Zero").

elif x == 1 do
    print("Single").

else do
    print("More").
    
end if

'''
interp(program, tree_dump=False, do_walk=True, exceptions=False)

More


In [63]:
program = \
'''
load "standard".
load "io".
load "util".

let x = 1.
while x <= 10 do
    print x.
    let x = x + 1.
end while
assert (x == 11).
'''
interp(program, exceptions=True)

1
2
3
4
5
6
7
8
9
10


In [64]:
program = \
'''
let 1 = 1.
'''
interp(program, exceptions=False, tree_dump=False)

In [65]:
program = \
'''
load "io".
load "util".

constructor A with arity 2.

let a = A(999, (lambda with none do return "Hello World!")).
print a@1 none.

assert(a@1 (none) is "Hello World!").

'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=True, do_walk=True)

Hello World!


In [66]:
program = \
'''
load "io".

print (1,2,3).
'''
interp(program)

(1,2,3)


In [67]:
program = \
'''
load "io".
load "util".
let nl = [[1 to 5]].
print nl.
assert (nl is [[1,2,3,4,5]]).
'''
interp(program, tree_dump=False, symtab_dump=False)

[[1,2,3,4,5]]


In [68]:
program = \
'''
-- TODO: Not yet implemented
-- let l = [2*i where i in [1 to 100]].
'''
interp(program, symtab_dump=False, tree_dump=False, exceptions=False)

In [69]:
program = \
'''
load "io".
load "util".
try 
    let [1 to 3] = [1,2,3].
catch (_, v) do
    print v.
    assert (v is "pattern match failed: pattern of type 'to-list' not allowed in pattern matching").
end try

let [1,2,3] = [1 to 3].
'''
interp(program, symtab_dump=False, tree_dump=False, exceptions=True)

pattern match failed: pattern of type 'to-list' not allowed in pattern matching


In [70]:
program = \
'''
load "io".
load "util".

constructor A with arity 3.

let a = A(1,2,3).
let b = a@[0 to 2].
print b.
assert (b is [1,2,3]).
'''
interp(program, symtab_dump=False, tree_dump=False, exceptions=True)

[1,2,3]


In [71]:
program = \
'''
load "io".
load "util".

let a = [0,1,2,3].
let b = a@[0 to 3 step 2].
print b.
assert (b is [0,2]).
'''
interp(program, symtab_dump=False, tree_dump=False, exceptions=True)

[0,2]


In [72]:
program = \
'''
load "io".
load "util".

let y = [0 to 10 step 2].
let z = 4 in y.
print z.
assert z.
'''
interp(program, symtab_dump=False, tree_dump=False)

true


In [73]:
program = \
'''
let true = 3 in [1,2,3].
'''
interp(program, symtab_dump=False)

In [74]:
program = \
'''
let true = (1,2) is (1,x).
let true = (1,3) is (1,x).

'''
interp(program, symtab_dump=False)

In [75]:
program =\
'''
load "io".
load "util".

constructor A with arity 2.

let foo = A(("a",1),("b",2)).
let foo@{"a"} = 2.

print foo@0.
assert (foo@0 is ("a",2)).
'''
interp(program, tree_dump=False, exceptions=False, symtab_dump=False)

(a,2)


In [76]:
program = \
'''
load "io".
load "util".

let d = [("a", 100)].
let d@{"foo"} = 1.
let d@{"goo"} = 2.

print d.
print d@{"a"}.
assert (d is [("a",100),("foo",1),("goo",2)]).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False)

[(a,100),(foo,1),(goo,2)]
100


In [77]:
program = \
'''
load "io".
load "util".

let b = [[1,2,3],
         [4,5,6],
         [7,8,9]].
let b@1@1 = 0.
print b.
assert (b is [[1,2,3],[4,0,6],[7,8,9]]).
'''
interp(program, tree_dump=False, exceptions=False)

[[1,2,3],[4,0,6],[7,8,9]]


In [78]:
program =\
'''
load "io".
load "util".

constructor A with arity 2.

let foo = A(("a",1),("b",2)).
print (foo@{"a"}).
assert (foo@{"a"} is 1).

let goo = A((1,1),(2,2)).
print (goo@{1}).
assert (goo@{1} is 1).
'''
interp(program, tree_dump=False, exceptions=True)

1
1


In [79]:
program = \
'''
let -1 = -1.
let not true = not true.
'''
interp(program, tree_dump=False, symtab_dump=False)

In [80]:
program = \
'''
-- infix operators are just a shorthand for prefix terms 
-- using special names for the operators


let 1 + 1 = __plus__ (1,1).
'''
interp(program, tree_dump=False, symtab_dump=False)

In [81]:
program = \
'''
load "io".
load "util".

constructor A with arity 1.
let a = A[1,2,3].
let a@0@1 = 0.
print a.
assert (a is A([1,0,3])).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False)

A([1,0,3])


In [82]:
program = \
'''
load "io".
load "util".

let a = [1,2,3].
let b = [0,0,0].
let [b@2,b@1,b@0] = a.
print b.
assert (b is [3,2,1]).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

[3,2,1]


In [83]:
program = \
'''
load "io".
load "util".

constructor A with arity 1.
constructor B with arity 2.

let x = A(1,).
let y = B(1,2).

let A(z,) = x.
let B(v,w) = y.

let xx = x@0.
let yy = y@[0,1].

print (x,y,xx,yy).
assert ((x,y,xx,yy) is (A(1,),B(1,2),1,[1,2])).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False)

(A(1,),B(1,2),1,[1,2])


In [84]:
program = \
'''
load "io".

let v = 'a@[3].
raw_print v.
'''
interp(program)

('quote',
 ('structure-ix',
  ('id', 'a'),
  ('index-list', ('list', [('index', ('list', [('integer', 3)]))]))))


In [85]:
program = \
'''
load "standard".
load "io".
load "util".

function inc with n do return n+1 end function

let v = inc(inc(0)).
let q = 1 + 1 + 1.
print (v, q).
assert ((v, q) is (2,3)).
'''
interp(program, tree_dump=False, symtab_dump=False)

(2,3)


In [86]:
constr = \
'''
-- constructors and pattern matching

load "standard".
load "io".
load "util".

constructor S with arity 1.
attach (lambda with n do return 1+n) to S.

let v = 'S(S(0)).
print v.
assert (v is S(S(0))).

let v = S(S(0)).
print v.
assert (v == 2).

let S (S (v))  = 'S(S(0)).
print v.
assert (v is 0).

let S (S (0))  = 'S(S(0)).


let [[v]] = [[0]].
print v.
assert (v is 0).

'''
interp(constr, tree_dump=False, symtab_dump=False, exceptions=False)

'S(S(0))
2
0
0


In [87]:
program = \
'''
-- show off our overloaded '+' operator
load "standard".
load "io".
load "util".

print (1 + 1).
assert ((1+1) == 2).

let s1 = "hello".
let s2 = "world".
let s3 = s1 + " " + s2 +"!".
print s3.
assert (s3 == "hello world!").

let l1 = [1,2,3].
let l2 = [4,5,6].
let l3 = l1 + l2.
print l3.
assert (l3 == [1,2,3,4,5,6]).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False)

2
hello world!
[1,2,3,4,5,6]


In [88]:
program = \
'''
load "io".
load "util".

-- reverse the list
let a = [1,2,3].
let a = a@[2,1,0].
print a.
assert (a is [3,2,1]).
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False, exceptions=False)

[3,2,1]


In [89]:
program = \
'''
load "io".
load "util".

-- reverse the list
let a = [1,2,3].
let a = a@[2,1,0].
print a.
assert (a is [3,2,1]).

-- access multidim array
let b = [[1,2,3],
         [4,5,6],
         [7,8,9]].
let e = b@1@1.
print e.
assert (e is 5).
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False, exceptions=False)

[3,2,1]
5


In [90]:
program = \
'''
-- the 'add' function implements the behavior for 
-- the '+' operator for ints, reals, and strings.

load "io".
load "util".

function add with (a, b) do return escape 
"
###################################################################
# return register from 'escape' calls
global __retval__ 
from asteroid_support import promote

# 'promote' is the type promotion table for primitive
# builtin types which implement the type hierarchy:
#    integer < real < string
type = promote(state.symbol_table.lookup_sym('a')[0], 
               state.symbol_table.lookup_sym('b')[0])

# select the correct add according to type
if type == 'integer':
    __retval__ = ('integer', 
            int(state.symbol_table.lookup_sym('a')[1]) + 
            int(state.symbol_table.lookup_sym('b')[1]))

elif type == 'real':
    __retval__ = ('real', 
            float(state.symbol_table.lookup_sym('a')[1]) + 
            float(state.symbol_table.lookup_sym('b')[1]))

elif type == 'string':
    __retval__ = ('string', 
            str(state.symbol_table.lookup_sym('a')[1])+
            str(state.symbol_table.lookup_sym('b')[1]))

else:
    raise ValueError('unsupported type in add')
###################################################################
"     
end function

-- attach the behavior to the '+' constructor
attach add to __plus__.

-- test the '+' operator with different data types
let x = 1.2 + 2 .
print x.
assert (x == 3.2).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False)

3.2


In [91]:
interp('load "io". load "util". let x + y = 1 + 2. print (x,y). assert ((x,y) is (1,2)).')

(1,2)


In [92]:

interp('load "io". load "util". let x = 1 + 2. print x. assert(x is __plus__(1,2))')

__plus__(1,2)


In [93]:
interp('load "io". load "util". let x = 1.3 . print x. assert(x is 1.3).', exceptions=False)

1.3


In [94]:
program =\
'''
load "io".
load "util".

try
    let 1 = 1.0 .
catch (_,v) do
    print v.
    assert(v is "pattern match failed: nodes 'real' and 'integer' are not the same").
end try
'''

interp(program)

pattern match failed: nodes 'real' and 'integer' are not the same


In [95]:
interp('load "io". load "util". let (1,y) = (1,2). print (1,y). assert((1,y) is (1,2)).')

(1,2)


In [96]:
interp('load "io". load "util". let (x,2) = (1,2). print (x,2). assert((x,2) is (1,2)).')

(1,2)


In [97]:
interp('load "io". load "util". let s = (1,2). let (x,y) = s. print (x,y). assert((x,y) is (1,2))')

(1,2)


In [98]:
program = \
'''
load "io".
load "util".

function ident with n do return n end function.
print (ident(2)).
assert (ident(2) is 2).
'''
interp(program)

2


In [99]:
program = \
'''
load "io".
load "util".
let x = 1. 
function ident with n do return n end function.
print (ident( ident (x))) .
assert (ident( ident (x)) is 1) .
'''
interp(program)

1


In [100]:
interp('load "util". let (_, x) = ([1], 2). assert (x is 2).', symtab_dump=False)

In [101]:
program = \
'''
load "standard"
load "io".
load "util".

print ((lambda with n do return n+1) 1).
assert ((lambda with n do return n+1) 1 is 2).
'''
interp(program, tree_dump=False, symtab_dump=False)

2
